In [27]:
import pandas as pd
import numpy as np
import re 
import sys
import os 
import json
from typing import Dict, List, Optional
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer

In [28]:
%load_ext autoreload
%autoreload 2
sys.path.insert(0, os.path.abspath('..'))
from recipe_gpt.preprocessing_utilities.preprocessing_functions import (check_nan_columns, 
                                                                        extract_nutritional_numerical)

In [29]:
path = '/home/victor/Documents/recipe_gpt/output/raw_recipes/df_final_7000_with_ingredients_fixed.csv'
df_recipes = pd.read_csv(path, index_col=0, sep='|')

## Clear columns 

#### Fix price 

In [30]:
fixed_prices = df_recipes['price'].apply(lambda x: extract_nutritional_numerical(x))

In [52]:
mask = fixed_prices == 2.5
print(f"To process: {sum(mask)}")

To process: 16


In [53]:
df_recipes.loc[mask, 'price']

395     2.5
595     2.5
663     2.5
1095    2.5
2322    2.5
2967    2.5
5688    2.5
6555    2.5
6735    2.5
6818    2.5
6839    2.5
6858    2.5
6870    2.5
6930    2.5
6993    2.5
6996    2.5
Name: price, dtype: object

In [50]:
fixed_prices[mask] = 3.0

In [51]:
fixed_prices.value_counts()

price
2.0    6334
1.0     601
3.0      44
2.5      16
Name: count, dtype: int64

#### Clean titles

In [22]:
mask = df_recipes['title'].str.contains('(', regex=False)
print(f"Total recipes with pattern {sum(mask)} ")

Total recipes with pattern 905 


In [23]:
df_to_fix = df_recipes.loc[mask, :]

In [24]:
mask_1 = df_to_fix['title']

485                           Aloo Tikki (Potato Patties)
521         Vegan Energy Balls (Dates, Nuts, and Coconut)
1110                    Vegan Thai Coconut Soup (Tom Kha)
1349      Lentil Curry (200 calories/portion, 4 portions)
1350    Mediterranean Quinoa Salad (250 calories/porti...
                              ...                        
6866    Persian Zucchini Stew With Chicken (Khoresh Ka...
6881                    Sautéed Tofu (Dau Hu Xao Dau Hao)
6899                   (Korean Black Bean Noodles) Recipe
6971     Russian Potato Salad Olivier (From South Russia)
6976                        Rolled Oats Master Mix (Oamc)
Name: title, Length: 905, dtype: object

In [19]:
df_to_fix['title_fixed'] = df_to_fix['title'].apply(lambda x: x.replace('', ''))

/tmp/ipykernel_275380/1177389703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_fix['title_fixed'] = df_to_fix['title'].apply(lambda x: x.replace('**', ''))


In [20]:
df_recipes.loc[mask, 'title'] = df_to_fix['title_fixed']

In [21]:
out_path = '/home/victor/Documents/recipe_gpt/output/raw_recipes/df_final_7000_normalized.csv'
df_recipes.to_csv(out_path, sep='|', index=True)